In [1]:
import pandas as pd
import boto3
import json

In [2]:
# --- Configuration ---
BUCKET_NAME = "mayank-rearc-quest-2025"
TIME_SERIES_KEY = "pr/pr.data.0.Current"
POPULATION_KEY = "population/us_population_data.json"

print("Configuration loaded.")

Configuration loaded.


In [3]:
# --- Load Data from S3 ---
s3_client = boto3.client('s3')

# Load the time-series data from the BLS files
try:
    print(f"Loading time-series data from s3://{BUCKET_NAME}/{TIME_SERIES_KEY}...")
    ts_obj = s3_client.get_object(Bucket=BUCKET_NAME, Key=TIME_SERIES_KEY)
    
    df_series = pd.read_csv(ts_obj['Body'], sep='\t')
    print("Time-series data loaded successfully.")

    # Load the population data from the JSON file
    print(f"Loading population data from s3://{BUCKET_NAME}/{POPULATION_KEY}...")
    pop_obj = s3_client.get_object(Bucket=BUCKET_NAME, Key=POPULATION_KEY)
    population_data = json.loads(pop_obj['Body'].read().decode('utf-8'))
    df_population = pd.DataFrame(population_data['data'])
    print("Population data loaded successfully.")
    
except Exception as e:
    print(f"An error occurred loading data from S3: {e}")

Loading time-series data from s3://mayank-rearc-quest-2025/pr/pr.data.0.Current...
Time-series data loaded successfully.
Loading population data from s3://mayank-rearc-quest-2025/population/us_population_data.json...
Population data loaded successfully.


In [4]:
# --- Data Cleaning ---
print("\n--- Starting Data Cleaning ---")

# Clean the time-series dataframe

# Trim whitespace from column headers
df_series.columns = df_series.columns.str.strip()

# Trim whitespace from the actual data in the 'series_id' and 'period' columns
df_series['series_id'] = df_series['series_id'].str.strip()
df_series['period'] = df_series['period'].str.strip()

# Convert the 'value' column to a number, forcing any non-numeric values to become 'NaN'
df_series['value'] = pd.to_numeric(df_series['value'], errors='coerce')

# Drop any rows that had non-numeric values, as they can't be used in calculations.
df_series.dropna(subset=['value'], inplace=True)
print("Time-series dataframe cleaned.")

# Clean the population dataframe
df_population['Year'] = df_population['Year'].astype(int)
print("Population dataframe cleaned.")
print("--- Data Cleaning Complete ---")


--- Starting Data Cleaning ---
Time-series dataframe cleaned.
Population dataframe cleaned.
--- Data Cleaning Complete ---


In [5]:
# --- Answering the Questions ---
print("\n--- Analytics Results ---")

# Question 1: What is the mean and standard deviation of the US population from 2013 to 2018?
print("\n[Question 1: US Population Analysis (2013-2018)]")
pop_filtered = df_population[(df_population['Year'] >= 2013) & (df_population['Year'] <= 2018)]
pop_mean = pop_filtered['Population'].mean()
pop_std = pop_filtered['Population'].std()
print(f"Mean Population: {pop_mean:,.0f}")
print(f"Standard Deviation of Population: {pop_std:,.0f}")


--- Analytics Results ---

[Question 1: US Population Analysis (2013-2018)]
Mean Population: 322,069,808
Standard Deviation of Population: 4,158,441


In [6]:
# Question 2: For each series_id, what was the year with the maximum sum of "value"?
print("\n[Question 2: Best Year per Series ID]")
yearly_sum = df_series.groupby(['series_id', 'year'])['value'].sum().reset_index()

# For each 'series_id', find the index of the row with the maximum 'value'
max_value_idx = yearly_sum.groupby('series_id')['value'].idxmax()
best_year_report = yearly_sum.loc[max_value_idx]
print("Top 5 results:")
print(best_year_report.head())


[Question 2: Best Year per Series ID]
Top 5 results:
       series_id  year    value
27   PRS30006011  2022   20.500
58   PRS30006012  2022   17.100
65   PRS30006013  1998  705.895
108  PRS30006021  2010   17.700
139  PRS30006022  2010   12.400


In [7]:
# Question 3: Join the two datasets to get the population for a specific series_id and period.
print("\n[Question 3: Joined Data for PRS30006032, Period Q01]")
# Filter for the specific series and period Q01
series_filtered = df_series[
    (df_series['series_id'] == 'PRS30006032') &
    (df_series['period'] == 'Q01')
].copy()

# Join the filtered data with the population data on the 'year' column
joined_df = pd.merge(
    series_filtered,
    df_population,
    left_on='year',
    right_on='Year',
    how='inner'
)
# Select and display the final desired columns
final_report = joined_df[['series_id', 'year', 'period', 'value', 'Population']]
print(final_report)

print("\n--- Analysis Complete ---")


[Question 3: Joined Data for PRS30006032, Period Q01]
     series_id  year period  value   Population
0  PRS30006032  2013    Q01    0.5  316128839.0
1  PRS30006032  2014    Q01   -0.1  318857056.0
2  PRS30006032  2015    Q01   -1.7  321418821.0
3  PRS30006032  2016    Q01   -1.4  323127515.0
4  PRS30006032  2017    Q01    0.9  325719178.0
5  PRS30006032  2018    Q01    0.5  327167439.0
6  PRS30006032  2019    Q01   -1.6  328239523.0
7  PRS30006032  2021    Q01    0.7  331893745.0
8  PRS30006032  2022    Q01    5.3  333287562.0
9  PRS30006032  2023    Q01    0.3  334914896.0

--- Analysis Complete ---
